## 펑션 콜링(function calling) 작동 방식 이해하기 

In [3]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import FunctionCallingAgent
from llama_index.core.tools import FunctionTool
import numpy as np

llm=OpenAI(model="gpt-4o")

In [4]:
def add(a,b):
    """ 주어진 두 숫자를 더하고 결과를 출력합니다 """
    return a+b


def mul(a,b):
    """ 주어진 두 숫자를 곱하고 결과를 출력합니다 """
    return a*b

def div(a,b):
    """ 주어진 두 숫자 중 첫번 째 숫자를 두번 째 숫자로 나누고 결과를 출력합니다 """
    return a/b


In [5]:
at=FunctionTool.from_defaults(fn=add)
mt=FunctionTool.from_defaults(fn=mul)
dt=FunctionTool.from_defaults(fn=div)

In [6]:
agent_worker=FunctionCallingAgent.from_tools([at,mt,dt], 
                                            llm=llm, 
                                            verbose=True,
                                            allow_parallel_tool_calls=False
                                            )


In [7]:
response=agent_worker.chat("(77*2+2)를 78로 나눈 값을 계산해 줘")
print(response)

> Running step 5dad04ee-7217-4f64-85e4-2b70bef36007. Step input: (77*2+2)를 78로 나눈 값을 계산해 줘
Added user message to memory: (77*2+2)를 78로 나눈 값을 계산해 줘
=== Calling Function ===
Calling function: mul with args: {"a": 77, "b": 2}
=== Function Output ===
154
> Running step df5b2a9d-6737-4573-87a0-2d6c4ffc64ea. Step input: None
=== Calling Function ===
Calling function: add with args: {"a": 154, "b": 2}
=== Function Output ===
156
> Running step 9da4b6e5-f0b9-41c9-9123-0f115197c625. Step input: None
=== Calling Function ===
Calling function: div with args: {"a": 156, "b": 78}
=== Function Output ===
2.0
> Running step fc724638-7954-49c8-b87d-87b11a5fdf5b. Step input: None
=== LLM Response ===
\((77 \times 2 + 2)\)를 78로 나눈 값은 2.0입니다.
\((77 \times 2 + 2)\)를 78로 나눈 값은 2.0입니다.


## 펑션 콜링 실습 -외부 API를 활용하는 Function Calling Agent

In [1]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import FunctionCallingAgent
from llama_index.core.tools import FunctionTool
import numpy as np

llm=OpenAI(model="gpt-4o")

In [2]:
!pip install yfinance==0.2.55

import yfinance as yf


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#yfinance 버전 확인인
print(yf.__version__)

0.2.55


In [4]:
# 미국 주식 최신 종가 호출 
def get_stock_price_us(code):
        ticker = yf.Ticker(f"{code}")
        todays_data = ticker.history(period='1d')
        return round(todays_data['Close'].iloc[0], 2)

# 한국 주식 최신 종가 호출 
def get_stock_price_Korea(code):
        ticker = yf.Ticker(f"{code}.KS")
        todays_data = ticker.history(period='1d')
        return round(todays_data['Close'].iloc[0], 2)


In [ ]:
## 'Encountered error: 'Too Many Requests. Rate limited. Try after a while.' 에러가 뜰 경우 함수를 아래와 같이 정의합니다
!pip install curl_cffi
from curl_cffi import requests


# 미국 주식 최신 종가 호출 
def get_stock_price_us(code):
        session = requests.Session(impersonate="chrome")
        ticker = yf.Ticker(f"{code}", session=session)
        todays_data = ticker.history(period='1d')
        return round(todays_data['Close'].iloc[0], 2)

# 한국 주식 최신 종가 호출 
def get_stock_price_Korea(code):
        session = requests.Session(impersonate="chrome")
        ticker = yf.Ticker(f"{code}.KS", session=session)
        todays_data = ticker.history(period='1d')
        return round(todays_data['Close'].iloc[0], 2)


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# 도구 등록하기 
stock_k=FunctionTool.from_defaults(fn=get_stock_price_Korea)
stock_u=FunctionTool.from_defaults(fn=get_stock_price_us)


In [ ]:
# 에이전트 만들고 쿼리 실행하기 
agent_worker=FunctionCallingAgent.from_tools([stock_k, stock_u], 
                                            llm=llm, 
                                            verbose=True,
                                            allow_parallel_tool_calls=False
                                            )

In [11]:
response=agent_worker.chat("TESLA 최신 종가가 어떻게 돼?")


> Running step 402af3c6-3070-4956-99fd-99ba89be7e1e. Step input: TESLA 최신 종가가 어떻게 돼?
Added user message to memory: TESLA 최신 종가가 어떻게 돼?
=== Calling Function ===
Calling function: get_stock_price_us with args: {"code": "TSLA"}
=== Function Output ===
342.82
> Running step 0fa2ac27-07d9-465b-9e78-0c515fc31d87. Step input: None
=== LLM Response ===
TESLA의 최신 종가는 $342.82입니다.


In [58]:
response=agent_worker.chat("삼성전자의 최신 종가가 어떻게 돼?")


> Running step c8f6483a-35e9-482b-b96f-37c1289647fd. Step input: 삼성전자의 최신 종가가 어떻게 돼?
Added user message to memory: 삼성전자의 최신 종가가 어떻게 돼?
=== Calling Function ===
Calling function: get_stock_price_Korea with args: {"code": "005930"}
=== Function Output ===
(57300.0, '2025-05-15')
> Running step 8a78fe0a-9741-4e5a-90ef-1a24e8f18d6a. Step input: None
=== LLM Response ===
삼성전자의 최신 종가는 2025년 5월 15일 기준으로 57,300원입니다.


### 주식 최신 종가를 날짜 정보와 함께 알고 싶을 때 

In [51]:
# 종가 함수부터 아래 코드를 실행 
# 미국 주식 최신 종가를 날짜 정보와 함께 알고 싶을 때  
def get_stock_price_us(code):
        ticker = yf.Ticker(f"{code}")
        todays_data = ticker.history(period='1d')

        if not todays_data.empty:
              close_price = round(todays_data['Close'].iloc[0], 2)
              close_date = todays_data.index[0].strftime('%Y-%m-%d')
              return close_price, close_date
   
        else:
              return None, None
        

# 한국 주식 최신 종가를 날짜 정보와 함께 알고 싶을 때 
def get_stock_price_Korea(code):
        ticker = yf.Ticker(f"{code}.KS")
        todays_data = ticker.history(period='1d')

        if not todays_data.empty:
              close_price = round(todays_data['Close'].iloc[0], 2)
              close_date = todays_data.index[0].strftime('%Y-%m-%d')
              return close_price, close_date
   
        else:
              return None, None               


In [52]:
stock_k=FunctionTool.from_defaults(fn=get_stock_price_Korea)
stock_u=FunctionTool.from_defaults(fn=get_stock_price_us)

In [53]:
agent_worker=FunctionCallingAgent.from_tools([stock_k, stock_u], 
                                            llm=llm, 
                                            verbose=True,
                                            allow_parallel_tool_calls=False
                                            )

In [54]:
response=agent_worker.chat("TESLA 최신 종가가 어떻게 돼?")

> Running step 36f13b1c-d353-4d94-b1b5-7c69a5d582d6. Step input: TESLA 최신 종가가 어떻게 돼?
Added user message to memory: TESLA 최신 종가가 어떻게 돼?
=== Calling Function ===
Calling function: get_stock_price_us with args: {"code": "TSLA"}
=== Function Output ===
(342.82, '2025-05-15')
> Running step 4fab3d5a-c49c-45ee-b702-340363f1e5f6. Step input: None
=== LLM Response ===
TESLA의 최신 종가는 2025년 5월 15일 기준으로 $342.82입니다.


In [ ]:
response=agent_worker.chat("삼성전자 최신 종가가 어떻게 돼?")

> Running step 5982dd31-2743-4019-8ab5-7d4847c84f02. Step input: 삼성전자자 최신 종가가 어떻게 돼?
Added user message to memory: 삼성전자자 최신 종가가 어떻게 돼?
=== Calling Function ===
Calling function: get_stock_price_Korea with args: {"code": "005930"}
=== Function Output ===
(55300.0, '2025-04-18')
> Running step e845d9ac-7ebd-472b-929f-ccd10addfa20. Step input: None
=== LLM Response ===
삼성전자의 최신 종가는 2025년 4월 18일 기준으로 55,300원입니다.


## 펑션 콜링 실습 - Context-Augmented Function Calling Agent

In [ ]:
#데이터 다운로드 
import os
import urllib.parse
import requests
import re

urls=[
    "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch08/data/%5B%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%5D%EC%82%AC%EC%97%85%EB%B3%B4%EA%B3%A0%EC%84%9C_2022.pdf",
    "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch08/data/%5B%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%5D%EC%82%AC%EC%97%85%EB%B3%B4%EA%B3%A0%EC%84%9C_2023.pdf",
    "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch08/data/%5B%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%5D%EC%82%AC%EC%97%85%EB%B3%B4%EA%B3%A0%EC%84%9C_2024.pdf"
    ]

# 각 파일 다운로드
for url in urls:
    encoded_filename = url.split("/")[-1]  # URL에서 파일명 추출
    decoded_filename = urllib.parse.unquote(encoded_filename) # 한글 파일명 복원
    response = requests.get(url)
    if response.status_code == 200:
# 임시 파일명으로 저장
        temp_filename = "temp_download_file" + os.path.splitext(decoded_filename)[1] 
        with open(temp_filename, 'wb') as f:
            f.write(response.content)            
        os.rename(temp_filename, decoded_filename) # 파일명 변경
        print(f"완료: {decoded_filename} 다운로드 완료")
    else:
        print(f"오류: {url} 다운로드 실패 (상태 코드: {response.status_code})\n")


In [12]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.settings import Settings

Settings.llm = OpenAI(model="gpt-4o")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [13]:

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex  
)
from llama_index.core.tools import QueryEngineTool

In [14]:
# load data
S2024_docs = SimpleDirectoryReader(
    input_files=["./data/[삼성전자]사업보고서_2024.pdf"]
    ).load_data()
S2023_docs = SimpleDirectoryReader(
    input_files=["./data/[삼성전자]사업보고서_2023.pdf"]
    ).load_data()
S2022_docs = SimpleDirectoryReader(
    input_files=["./data/[삼성전자]사업보고서_2022.pdf"]
    ).load_data()

# build index
S2024_index = VectorStoreIndex.from_documents(S2024_docs)
S2023_index = VectorStoreIndex.from_documents(S2023_docs)
S2022_index = VectorStoreIndex.from_documents(S2022_docs)
    

In [15]:
S2024_engine = S2024_index.as_query_engine(similarity_top_k=3)
S2023_engine = S2023_index.as_query_engine(similarity_top_k=3)
S2022_engine = S2022_index.as_query_engine(similarity_top_k=3)

In [24]:
query_engine_tools = [
    QueryEngineTool.from_defaults(
        query_engine=S2024_engine,
        name="samsung_2024",
        description=(
            "삼성전자의 2024년 재무상태에 대해 정보 제공해 주세요."
            "도구에 입력할 때 자세한 일반 텍스트 질문을 사용합니다."
            "실적 분석할 때 보고서를 충분히 검토한 후 답변해 주세요"
        ),
    ),
    QueryEngineTool.from_defaults(
        query_engine=S2023_engine,
        name="samsung_2023",
        description=(
             "삼성전자의 2023년 재무상태에 대해 정보 제공해 주세요."
            "도구에 입력할 때 자세한 일반 텍스트 질문을 사용합니다."
            "실적 분석할 때 보고서를 충분히 검토한 후 답변해 주세요"
        ),
    ),
    QueryEngineTool.from_defaults(
        query_engine=S2022_engine,
        name="samsung_2022",
        description=(
             "삼성전자의 2022년 재무상태에 대해 정보 제공해 주세요."
            "도구에 입력할 때 자세한 일반 텍스트 질문을 사용합니다."
            "실적 분석할 때 보고서를 충분히 검토한 후 답변해 주세요"
        ),
    ),
]

In [25]:
# agent 생성
from llama_index.core.agent import FunctionCallingAgent

agent = FunctionCallingAgent.from_tools(tools=query_engine_tools,  # tools 대신 query_engine_tools 사용
                                               llm=OpenAI(model="gpt-4o"),
                                               verbose=True,
                                               )


In [37]:
response=agent.chat("2022년 매출액과 2023년 매출액, 2024년 매출액을 차례로 알려줘")


> Running step 8f7269ce-0b55-4d08-b11b-77afd60b8f7c. Step input: 2022년 매출액과 2023년 매출액, 2024년 매출액을 차례로 알려줘
Added user message to memory: 2022년 매출액과 2023년 매출액, 2024년 매출액을 차례로 알려줘
=== Calling Function ===
Calling function: samsung_2022 with args: {"input": "2022\ub144 \uc0bc\uc131\uc804\uc790\uc758 \ub9e4\ucd9c\uc561\uc744 \uc54c\ub824\uc8fc\uc138\uc694."}
=== Function Output ===
2022년 삼성전자의 매출액은 302조 2,314억원입니다.
=== Calling Function ===
Calling function: samsung_2023 with args: {"input": "2023\ub144 \uc0bc\uc131\uc804\uc790\uc758 \ub9e4\ucd9c\uc561\uc744 \uc54c\ub824\uc8fc\uc138\uc694."}
=== Function Output ===
2023년 삼성전자의 매출액은 258조 9,355억원입니다.
=== Calling Function ===
Calling function: samsung_2024 with args: {"input": "2024\ub144 \uc0bc\uc131\uc804\uc790\uc758 \ub9e4\ucd9c\uc561\uc744 \uc54c\ub824\uc8fc\uc138\uc694."}
=== Function Output ===
2024년 삼성전자의 매출액은 300조 8,709억원입니다.
> Running step 75a394b2-cb51-431a-8d30-845e149498e0. Step input: None
=== LLM Response ===
- 2022년 삼성전자의 매출액은 30

In [18]:
response=agent.chat("삼성전자의 2022년, 2023년, 2024년 매출액과 영업이익을 알려줘. 그리고 2023년에는 전년대비 줄어든 매출액이 2024년에는 다시 회복된 이유를 뭐라고 분석하고 있는지 찾아줘")

> Running step 1cc5bf67-732e-4dbb-afcc-01690826f851. Step input: 삼성전자의 2022년, 2023년, 2024년 매출액과 영업이익을 알려줘. 그리고 2023년에는 전년대비 줄어든 매출액이 2024년에는 다시 회복된 이유를 뭐라고 분석하고 있는지 찾아줘
Added user message to memory: 삼성전자의 2022년, 2023년, 2024년 매출액과 영업이익을 알려줘. 그리고 2023년에는 전년대비 줄어든 매출액이 2024년에는 다시 회복된 이유를 뭐라고 분석하고 있는지 찾아줘
=== Calling Function ===
Calling function: samsung_2022 with args: {"input": "\uc0bc\uc131\uc804\uc790\uc758 2022\ub144 \ub9e4\ucd9c\uc561\uacfc \uc601\uc5c5\uc774\uc775\uc744 \uc54c\ub824\uc918"}
=== Function Output ===
삼성전자의 2022년 매출액은 302조 2,314억 원이며, 영업이익은 51,633,856백만 원입니다.
=== Calling Function ===
Calling function: samsung_2023 with args: {"input": "\uc0bc\uc131\uc804\uc790\uc758 2023\ub144 \ub9e4\ucd9c\uc561\uacfc \uc601\uc5c5\uc774\uc775\uc744 \uc54c\ub824\uc918"}
=== Function Output ===
삼성전자의 2023년 매출액은 258조 9,355억 원이며, 영업이익은 6조 5,669억 원입니다.
=== Calling Function ===
Calling function: samsung_2024 with args: {"input": "\uc0bc\uc131\uc804\uc790\uc758 2024\ub144 \ub9e4\ucd9c\uc561\u

In [26]:
response=agent.chat("삼성전자의 2023년 매출액이 2022년에 비해 많이 줄었는데, 2024년에는 다시 회복됐어. 이유를 뭐라고 분석하고 있어? 보고서를 충분히 검토하고 답변해 줘")

> Running step 146f0802-b8ac-427a-9356-3106de9093ab. Step input: 삼성전자의 2023년 매출액이 2022년에 비해 많이 줄었는데, 2024년에는 다시 회복됐어. 이유를 뭐라고 분석하고 있어? 보고서를 충분히 검토하고 답변해 줘
Added user message to memory: 삼성전자의 2023년 매출액이 2022년에 비해 많이 줄었는데, 2024년에는 다시 회복됐어. 이유를 뭐라고 분석하고 있어? 보고서를 충분히 검토하고 답변해 줘
=== Calling Function ===
Calling function: samsung_2023 with args: {"input": "\uc0bc\uc131\uc804\uc790\uc758 2023\ub144 \ub9e4\ucd9c\uc561\uc774 2022\ub144\uc5d0 \ube44\ud574 \ub9ce\uc774 \uc904\uc5b4\ub4e0 \uc774\uc720\uc5d0 \ub300\ud574 \ubd84\uc11d\ud574 \uc8fc\uc138\uc694."}
=== Function Output ===
2023년 삼성전자의 매출액은 전년 대비 14.3% 감소했습니다. 부문별로 보면, DX 부문은 6.8%, DS 부문은 32.4%, SDC는 9.9% 감소한 반면, Harman은 8.9% 증가했습니다. 특히 DS 부문의 매출 감소가 두드러지며, 이는 메모리 제품의 평균 판매가격이 전년 대비 약 45% 하락한 것이 주요 원인 중 하나로 보입니다. 또한, TV의 평균 판매가격도 약 3% 하락하여 매출 감소에 영향을 미쳤습니다. 반면, 스마트폰과 스마트폰용 OLED 패널의 가격은 상승했으나, 전체 매출 감소를 상쇄하기에는 부족했던 것으로 보입니다.
=== Calling Function ===
Calling function: samsung_2024 with args: {"input": "\uc0bc\uc131\uc804\uc790\uc758 2024\u